In [1]:
from sys import version
import tensorflow
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

print(version)

Using TensorFlow backend.
3.6.2 |Anaconda custom (64-bit)| (default, Sep 21 2017, 18:29:43) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
# Load and prepare training and xval data
TRAINING_FILE, XVAL_FILE = "../combined_data/15min/train.tsv.gz", "../combined_data/15min/xval.tsv.gz"
train, xval = pd.read_csv(TRAINING_FILE, sep='\t'), pd.read_csv(XVAL_FILE, sep='\t')
print(f'Training dimension: {train.shape}')
print(f'Xval dimension: {xval.shape}')
print('\n'.join(train.columns))
train.head(n=2).T

Training dimension: (542954, 20)
Xval dimension: (67876, 20)
opd_date
trip_start_hr_15
rte
dir
day_of_week
is_ns
is_rapid
is_weekend
orca_total
frac_disabled
frac_youth
frac_senior
frac_li
frac_uw
ons
region
start
end
type
summer


,0,1
opd_date,2019-01-07,2019-01-07
trip_start_hr_15,00_0,00_0
rte,1,2
dir,N,N
day_of_week,0,0
is_ns,1,1
is_rapid,0,0
is_weekend,0,0
orca_total,2,3
frac_disabled,0.0666667,0.00641026


In [3]:

X_NUM_COLS = [
    'orca_total', 'frac_disabled', 'frac_youth', 'frac_senior', 'frac_li', 'frac_uw'
]
X_CAT_COLS = ['is_ns', 'is_rapid', 'is_weekend', 'trip_start_hr_15', 'rte', 'dir', 'day_of_week', 'region', 'start', 'end', 'summer']


label_encoders = {col: LabelEncoder() for col in X_CAT_COLS}
one_hot_encoder = OneHotEncoder()
scaler = StandardScaler()

X_train = np.concatenate((
    scaler.fit_transform(train[X_NUM_COLS]),
    one_hot_encoder.fit_transform(
        np.stack([label_encoders[col].fit_transform(train[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)
X_xval = np.concatenate((
    scaler.transform(xval[X_NUM_COLS]),
    one_hot_encoder.transform(
        np.stack([label_encoders[col].transform(xval[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)

y_train = train['ons']
y_xval = xval['ons']

column_labels = list()
column_labels.extend(X_NUM_COLS)
for cat in X_CAT_COLS:
    for clazz in label_encoders[cat].classes_:
        column_labels.append(f'{cat}: {clazz}')

In [4]:
assert len(column_labels) == X_train.shape[1]

for i, label in enumerate(column_labels):
    print(f'{label}: {np.squeeze(X_train[:5, i])}')

orca_total: [[-0.76657383 -0.72243036 -0.45756958 -0.85486075 -0.6782869 ]]
frac_disabled: [[ 1.36376967 -0.41035794 -0.54689108 -0.59909492  0.01430026]]
frac_youth: [[-0.50030876 -0.50030876 -0.50030876 -0.50030876 -0.50030876]]
frac_senior: [[-0.53708954 -0.53708954 -0.53708954 -0.53708954 -0.53708954]]
frac_li: [[-0.65231668 -0.30054282 -0.31176964 -0.65231668 -0.65231668]]
frac_uw: [[-0.4245457  -0.4245457  -0.2131194  -0.4245457  -0.01050253]]
is_ns: 0.0: [[0. 0. 0. 0. 0.]]
is_ns: 1.0: [[1. 1. 1. 1. 1.]]
is_rapid: 0.0: [[1. 1. 1. 1. 1.]]
is_rapid: 1.0: [[0. 0. 0. 0. 0.]]
is_weekend: 0.0: [[1. 1. 1. 1. 1.]]
is_weekend: 1.0: [[0. 0. 0. 0. 0.]]
trip_start_hr_15: 00_0: [[1. 1. 1. 1. 1.]]
trip_start_hr_15: 00_15: [[0. 0. 0. 0. 0.]]
trip_start_hr_15: 00_30: [[0. 0. 0. 0. 0.]]
trip_start_hr_15: 00_45: [[0. 0. 0. 0. 0.]]
trip_start_hr_15: 01_0: [[0. 0. 0. 0. 0.]]
trip_start_hr_15: 01_15: [[0. 0. 0. 0. 0.]]
trip_start_hr_15: 01_30: [[0. 0. 0. 0. 0.]]
trip_start_hr_15: 01_45: [[0. 0. 0. 0.

In [27]:
# Attempt 1: long thin neural net
xval_perf = list()
for n_middle_layers in (1,2,4,8,10):
    print(f"{n_middle_layers} middle layers")
    model = Sequential()
    model.add(Dense(4, activation="linear", input_dim=426))
    model.add(Dropout(0.5))
    for i in range(n_middle_layers-1):
        model.add(Dense(4, activation='sigmoid'))
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
    model.fit(X_train, y_train, epochs=10, batch_size=256)
    xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
    xval_perf.append(xval_mae)
    print(xval_mae)

1 middle layers
Epoch 1/10
542954/542954 [==============================] - 9s 17us/step - loss: 13.4350 - mean_absolute_error: 13.4350 - mean_squared_error: 586.5932
Epoch 2/10
542954/542954 [==============================] - 7s 14us/step - loss: 13.2179 - mean_absolute_error: 13.2179 - mean_squared_error: 571.8288
Epoch 3/10
542954/542954 [==============================] - 7s 14us/step - loss: 13.2228 - mean_absolute_error: 13.2228 - mean_squared_error: 574.3151
Epoch 4/10
542954/542954 [==============================] - 7s 14us/step - loss: 13.2007 - mean_absolute_error: 13.2007 - mean_squared_error: 567.6453
Epoch 5/10
542954/542954 [==============================] - 8s 14us/step - loss: 13.2090 - mean_absolute_error: 13.2090 - mean_squared_error: 572.9196
Epoch 6/10
542954/542954 [==============================] - 8s 14us/step - loss: 13.2216 - mean_absolute_error: 13.2216 - mean_squared_error: 571.7853
Epoch 7/10
542954/542954 [==============================] - 8s 14us/step - los

In [28]:
xval_perf   # 9.86 is pretty good

[9.860205568023265,
 15.237501715748275,
 17.95892187204293,
 22.89282713942961,
 22.887278550135814]

In [46]:
# transfer learning approach. Train a neural net to classify up to n clusters.
from sklearn.cluster import KMeans
from numpy.random import uniform, seed

n_clusters = 24
# Best I could do at 6: 21 mae on training

seed(10)
X_samp = X_train[uniform(size=X_train.shape[0]) < 0.1,:]
kmeans = KMeans(n_clusters, n_jobs=-1)
cluster_preds = kmeans.fit_predict(X_samp)
X_classes = kmeans.predict(X_train)

In [47]:
pd.Series(X_classes).value_counts()

3     44631
13    39301
1     39136
19    37222
0     33689
4     33535
12    31647
14    31437
17    30908
9     30571
11    28349
16    22873
6     20808
2     20480
20    19412
5     15292
22    11544
15     9966
18     9062
7      8623
8      7331
23     6517
21     5576
10     5044
dtype: int64

In [49]:
from keras.utils import np_utils

classifier = Sequential()
classifier.add(Dense(32, activation="relu", input_dim=X_train.shape[1]))
classifier.add(Dropout(0.5))
classifier.add(Dense(n_clusters, activation='sigmoid'))
classifier.add(Dropout(0.5))
classifier.add(Dense(n_clusters, activation='sigmoid'))
classifier.add(Dropout(0.5))
classifier.add(Dense(n_clusters, activation='softmax'))

classifier.compile(
    loss='binary_crossentropy', 
    #optimizer='adam',
    optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
classifier.fit(
    X_train, 
    np_utils.to_categorical(X_classes), 
    epochs=5, 
    batch_size=256)

Epoch 1/5
542954/542954 [==============================] - 33s 62us/step - loss: 0.1678 - acc: 0.9583
Epoch 2/5
542954/542954 [==============================] - 31s 57us/step - loss: 0.1623 - acc: 0.9583
Epoch 3/5
542954/542954 [==============================] - 31s 56us/step - loss: 0.1469 - acc: 0.9583
Epoch 4/5
542954/542954 [==============================] - 31s 57us/step - loss: 0.1357 - acc: 0.9583
Epoch 5/5
542954/542954 [==============================] - 31s 57us/step - loss: 0.1309 - acc: 0.9584


In [50]:

# predictor = Sequential()
# predictor.add(Dense(4, activation="relu", input_dim=n_clusters))
# predictor.add(Dropout(0.5))
# predictor.add(Dense(4, activation="sigmoid"))
# predictor.add(Dropout(0.5))
# predictor.add(Dense(4, activation="sigmoid"))
# predictor.add(Dropout(0.5))
# predictor.add(Dense(1, activation="linear"))
# 
# predictor.compile(
#     loss='mean_absolute_error', 
#     optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True), 
#     metrics=['mean_squared_error']
# )

In [52]:
#_train_fwd = classifier.predict(X_train)  # (n, n_clusters)
#predictor.fit(X_train_fwd, y_train, epochs=15, batch_size=256)

In [53]:
X_train_nums = X_train[:,:len(X_NUM_COLS)]
X_train_fwd = np.concatenate([classifier.predict(X_train), X_train_nums], axis=1)  # (n, n_clusters + len(X_NUM_COLS))
print(X_train_fwd.shape)

(542954, 30)


In [55]:
# So that didn't work the way I wanted. Let's add the numeric variables back in addition to the classifier outputs
predictor = Sequential()
predictor.add(Dense(4, activation="relu", input_dim=X_train_fwd.shape[1]))
predictor.add(Dropout(0.3))
predictor.add(Dense(4, activation="tanh"))
predictor.add(Dropout(0.3))
predictor.add(Dense(2, activation="sigmoid"))
predictor.add(Dropout(0.3))
predictor.add(Dense(2, activation="sigmoid"))
predictor.add(Dropout(0.3))
predictor.add(Dense(1, activation="linear"))
predictor.compile(
    loss='mean_absolute_error', 
    optimizer=SGD(lr=0.05, decay=1e-5, momentum=0.9, nesterov=True), 
    metrics=['mean_squared_error']
)

predictor.fit(X_train_fwd, y_train, epochs=20, batch_size=256)

Epoch 1/20
542954/542954 [==============================] - 15s 27us/step - loss: 20.8099 - mean_squared_error: 1176.8815
Epoch 2/20
542954/542954 [==============================] - 12s 22us/step - loss: 19.8858 - mean_squared_error: 1075.9795
Epoch 3/20
542954/542954 [==============================] - 12s 22us/step - loss: 19.7547 - mean_squared_error: 1076.0079
Epoch 4/20
542954/542954 [==============================] - 12s 22us/step - loss: 19.7005 - mean_squared_error: 1075.1803
Epoch 5/20
542954/542954 [==============================] - 12s 22us/step - loss: 19.8009 - mean_squared_error: 1073.6226
Epoch 6/20
542954/542954 [==============================] - 12s 22us/step - loss: 19.8341 - mean_squared_error: 1080.8641
Epoch 7/20
542954/542954 [==============================] - 12s 22us/step - loss: 19.5889 - mean_squared_error: 1055.1840
Epoch 8/20
542954/542954 [==============================] - 12s 22us/step - loss: 19.5703 - mean_squared_error: 1052.4492
Epoch 9/20
542954/542954

In [56]:
forward = np.concatenate([classifier.predict(X_xval), X_xval[:,:len(X_NUM_COLS)]], axis=1)
predictor.evaluate(forward, y_xval, batch_size=128)[0]

67876/67876 [==============================] - 2s 23us/step


19.639980235682316

In [59]:
# Try a smaller predictor network
predictor = Sequential()
predictor.add(Dense(8, activation="linear", input_dim=X_train_fwd.shape[1]))
predictor.add(Dropout(0.4))
predictor.add(Dense(4, activation="tanh"))
predictor.add(Dropout(0.4))
predictor.add(Dense(1, activation="linear"))
predictor.compile(
    loss='mean_absolute_error', 
    optimizer=SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True), 
    metrics=['mean_squared_error']
)

predictor.fit(X_train_fwd, y_train, epochs=20, batch_size=256)

forward = np.concatenate([classifier.predict(X_xval), X_xval[:,:len(X_NUM_COLS)]], axis=1)
predictor.evaluate(forward, y_xval, batch_size=128)[0]

Epoch 1/20
542954/542954 [==============================] - 12s 22us/step - loss: 17.6996 - mean_squared_error: 945.0755
Epoch 2/20
542954/542954 [==============================] - 9s 17us/step - loss: 16.9535 - mean_squared_error: 873.0365
Epoch 3/20
542954/542954 [==============================] - 9s 17us/step - loss: 16.9322 - mean_squared_error: 872.8653
Epoch 4/20
542954/542954 [==============================] - 9s 17us/step - loss: 16.9046 - mean_squared_error: 872.2459
Epoch 5/20
542954/542954 [==============================] - 9s 17us/step - loss: 16.9202 - mean_squared_error: 872.5690
Epoch 6/20
542954/542954 [==============================] - 9s 17us/step - loss: 16.9241 - mean_squared_error: 873.3550
Epoch 7/20
542954/542954 [==============================] - 10s 18us/step - loss: 16.9030 - mean_squared_error: 871.6740
Epoch 8/20
542954/542954 [==============================] - 10s 18us/step - loss: 16.9024 - mean_squared_error: 869.1037
Epoch 9/20
542954/542954 [===========

14.663555800342216

In [61]:
# Next try again with only a few clusters (4)
# transfer learning approach. Train a neural net to classify up to n clusters.
from sklearn.cluster import KMeans
from numpy.random import uniform, seed
from keras.utils import np_utils

n_clusters = 4
# Best I could do at 6: 21 mae on training

seed(10)
print("K MEANS")
X_samp = X_train[uniform(size=X_train.shape[0]) < 0.1,:]
kmeans = KMeans(n_clusters, n_jobs=-1)
cluster_preds = kmeans.fit_predict(X_samp)
X_classes = kmeans.predict(X_train)

print("CLASSIFIER")
classifier = Sequential()
classifier.add(Dense(32, activation="relu", input_dim=X_train.shape[1]))
classifier.add(Dropout(0.5))
classifier.add(Dense(n_clusters, activation='sigmoid'))
classifier.add(Dropout(0.5))
classifier.add(Dense(n_clusters, activation='sigmoid'))
classifier.add(Dropout(0.5))
classifier.add(Dense(n_clusters, activation='softmax'))

classifier.compile(
    loss='binary_crossentropy', 
    #optimizer='adam',
    optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
classifier.fit(
    X_train, 
    np_utils.to_categorical(X_classes), 
    epochs=5, 
    batch_size=256)

X_train_nums = X_train[:,:len(X_NUM_COLS)]
X_train_fwd = np.concatenate([classifier.predict(X_train), X_train_nums], axis=1)  # (n, n_clusters + len(X_NUM_COLS))
print(X_train_fwd.shape)

print("PREDICTOR")
# Try a smaller predictor network
predictor = Sequential()
predictor.add(Dense(8, activation="linear", input_dim=X_train_fwd.shape[1]))
predictor.add(Dropout(0.4))
predictor.add(Dense(4, activation="tanh"))
predictor.add(Dropout(0.4))
predictor.add(Dense(1, activation="linear"))
predictor.compile(
    loss='mean_absolute_error', 
    optimizer=SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True), 
    metrics=['mean_squared_error']
)

predictor.fit(X_train_fwd, y_train, epochs=20, batch_size=256)

forward = np.concatenate([classifier.predict(X_xval), X_xval[:,:len(X_NUM_COLS)]], axis=1)
predictor.evaluate(forward, y_xval, batch_size=128)[0]

K MEANS
CLASSIFIER
Epoch 1/5
542954/542954 [==============================] - 24s 44us/step - loss: 0.3726 - acc: 0.8286
Epoch 2/5
542954/542954 [==============================] - 20s 38us/step - loss: 0.3236 - acc: 0.8516
Epoch 3/5
542954/542954 [==============================] - 21s 38us/step - loss: 0.3112 - acc: 0.8614
Epoch 4/5
542954/542954 [==============================] - 21s 38us/step - loss: 0.3086 - acc: 0.8620
Epoch 5/5
542954/542954 [==============================] - 21s 38us/step - loss: 0.3068 - acc: 0.8626
(542954, 10)
PREDICTOR
Epoch 1/20
542954/542954 [==============================] - 12s 22us/step - loss: 18.1369 - mean_squared_error: 1000.1300
Epoch 2/20
542954/542954 [==============================] - 9s 17us/step - loss: 17.5080 - mean_squared_error: 934.9683
Epoch 3/20
542954/542954 [==============================] - 9s 17us/step - loss: 17.4990 - mean_squared_error: 933.7224
Epoch 4/20
542954/542954 [==============================] - 9s 17us/step - loss: 17.50

15.65815137514376

In [77]:

model = Sequential()
model.add(Dense(64, activation="linear", input_dim=426))
model.add(Dropout(0.5))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, epochs=10, batch_size=256)

xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(xval_mae)

Epoch 1/10
542954/542954 [==============================] - 18s 33us/step - loss: 19.5799 - mean_absolute_error: 19.5799 - mean_squared_error: 1061.7573
Epoch 2/10
542954/542954 [==============================] - 15s 27us/step - loss: 15.4098 - mean_absolute_error: 15.4098 - mean_squared_error: 699.5145
Epoch 3/10
542954/542954 [==============================] - 15s 27us/step - loss: 14.7907 - mean_absolute_error: 14.7907 - mean_squared_error: 664.1485
Epoch 4/10
542954/542954 [==============================] - 15s 27us/step - loss: 14.7151 - mean_absolute_error: 14.7151 - mean_squared_error: 588.5226
Epoch 5/10
542954/542954 [==============================] - 15s 27us/step - loss: 14.4890 - mean_absolute_error: 14.4890 - mean_squared_error: 599.8654
Epoch 6/10
542954/542954 [==============================] - 15s 27us/step - loss: 14.0169 - mean_absolute_error: 14.0169 - mean_squared_error: 594.1250
Epoch 7/10
542954/542954 [==============================] - 15s 27us/step - loss: 14.14

In [78]:

model = Sequential()
model.add(Dense(64, activation="sigmoid", input_dim=426))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, epochs=5, batch_size=256)

xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(xval_mae)

Epoch 1/5
542954/542954 [==============================] - 18s 34us/step - loss: 10.0502 - mean_absolute_error: 10.0502 - mean_squared_error: 342.7076
Epoch 2/5
542954/542954 [==============================] - 15s 28us/step - loss: 9.3352 - mean_absolute_error: 9.3352 - mean_squared_error: 282.5130
Epoch 3/5
542954/542954 [==============================] - 15s 28us/step - loss: 9.1425 - mean_absolute_error: 9.1425 - mean_squared_error: 267.0279
Epoch 4/5
542954/542954 [==============================] - 15s 28us/step - loss: 9.0709 - mean_absolute_error: 9.0709 - mean_squared_error: 261.0028
Epoch 5/5
67876/67876 [==============================] - 2s 34us/step
7.448609103815916


In [79]:
# Try other activation functions
for act in ['relu', 'tanh', 'linear']:
    print(f"ACTIVATION: {act}")
    model = Sequential()
    model.add(Dense(64, activation=act, input_dim=426))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
    model.fit(X_train, y_train, epochs=5, batch_size=256)
    xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
    print(f"XVAL MAE: {xval_mae}")


ACTIVATION: relu
Epoch 1/5
542954/542954 [==============================] - 18s 32us/step - loss: 10.3834 - mean_absolute_error: 10.3834 - mean_squared_error: 358.2582
Epoch 2/5
542954/542954 [==============================] - 15s 27us/step - loss: 9.8253 - mean_absolute_error: 9.8253 - mean_squared_error: 312.9068
Epoch 3/5
542954/542954 [==============================] - 14s 26us/step - loss: 9.6647 - mean_absolute_error: 9.6647 - mean_squared_error: 300.0488
Epoch 4/5
542954/542954 [==============================] - 15s 27us/step - loss: 9.6160 - mean_absolute_error: 9.6160 - mean_squared_error: 297.7739
Epoch 5/5
67876/67876 [==============================] - 2s 34us/step
XVAL MAE: 8.229848937734253
ACTIVATION: tanh
Epoch 1/5
542954/542954 [==============================] - 18s 34us/step - loss: 11.8733 - mean_absolute_error: 11.8733 - mean_squared_error: 507.1493
Epoch 2/5
542954/542954 [==============================] - 15s 28us/step - loss: 10.9242 - mean_absolute_error: 10.9242

In [80]:
# Sigmoid is best. Experiment with different 1st layer sizes
for sz in (8, 64, 128, 256):
    print(f"1st layer size: {sz}")
    model = Sequential()
    model.add(Dense(sz, activation="sigmoid", input_dim=426))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
    model.fit(X_train, y_train, epochs=5, batch_size=256)
    xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
    print(f"XVAL MAE: {xval_mae}")


1st layer size: 8
Epoch 1/5
542954/542954 [==============================] - 13s 24us/step - loss: 14.0998 - mean_absolute_error: 14.0998 - mean_squared_error: 613.1158
Epoch 2/5
542954/542954 [==============================] - 9s 17us/step - loss: 13.6424 - mean_absolute_error: 13.6424 - mean_squared_error: 562.9750
Epoch 3/5
542954/542954 [==============================] - 9s 17us/step - loss: 13.2836 - mean_absolute_error: 13.2836 - mean_squared_error: 529.4608
Epoch 4/5
542954/542954 [==============================] - 9s 17us/step - loss: 13.2173 - mean_absolute_error: 13.2173 - mean_squared_error: 524.4100
Epoch 5/5
67876/67876 [==============================] - 2s 32us/step
XVAL MAE: 10.520857586001625
1st layer size: 64
Epoch 1/5
542954/542954 [==============================] - 19s 35us/step - loss: 10.0374 - mean_absolute_error: 10.0374 - mean_squared_error: 342.1434
Epoch 2/5
542954/542954 [==============================] - 16s 29us/step - loss: 9.3315 - mean_absolute_error: 9

In [81]:
# ok, let's try a few options smaller and larger of 256 larger. With 1 more epoch for an additional boost.
for sz in (200, 512, 600, 1024):
    print(f"1st layer size: {sz}")
    model = Sequential()
    model.add(Dense(sz, activation="sigmoid", input_dim=426))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
    model.fit(X_train, y_train, epochs=6, batch_size=256)
    xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
    print(f"XVAL MAE: {xval_mae}")


1st layer size: 200
Epoch 1/6
542954/542954 [==============================] - 31s 56us/step - loss: 9.3590 - mean_absolute_error: 9.3590 - mean_squared_error: 315.1599
Epoch 2/6
542954/542954 [==============================] - 27s 50us/step - loss: 8.3751 - mean_absolute_error: 8.3751 - mean_squared_error: 244.7735
Epoch 3/6
542954/542954 [==============================] - 28s 51us/step - loss: 8.1250 - mean_absolute_error: 8.1250 - mean_squared_error: 226.4966
Epoch 4/6
542954/542954 [==============================] - 26s 49us/step - loss: 7.9898 - mean_absolute_error: 7.9898 - mean_squared_error: 218.0753
Epoch 5/6
542954/542954 [==============================] - 26s 48us/step - loss: 7.9411 - mean_absolute_error: 7.9411 - mean_squared_error: 214.7887
Epoch 6/6
67876/67876 [==============================] - 3s 40us/step
XVAL MAE: 7.077427285457586
1st layer size: 512
Epoch 1/6
542954/542954 [==============================] - 43s 80us/step - loss: 9.2074 - mean_absolute_error: 9.2074

In [84]:
# So 600 is the best in class so far. 
# What about adding another layer now that I think the combinations are being represented?
    
model = Sequential()
model.add(Dense(600, activation="sigmoid", input_dim=426))
model.add(Dropout(0.5))
model.add(Dense(4, activation="linear"))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, epochs=6, batch_size=256)
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")

Epoch 1/6
423680/542954 [======================>.......] - ETA: 10s - loss: 16.8686 - mean_absolute_error: 16.8686 - mean_squared_error: 2435.3308

KeyboardInterrupt: 

In [85]:
# lol NOPE. What about less dropout?
model = Sequential()
model.add(Dense(600, activation="sigmoid", input_dim=426))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=a0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, epochs=6, batch_size=256)
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")

Epoch 1/6
542954/542954 [==============================] - 47s 87us/step - loss: 8.9467 - mean_absolute_error: 8.9467 - mean_squared_error: 305.1940
Epoch 2/6
542954/542954 [==============================] - 46s 85us/step - loss: 7.8276 - mean_absolute_error: 7.8276 - mean_squared_error: 226.5004
Epoch 3/6
542954/542954 [==============================] - 48s 88us/step - loss: 7.5184 - mean_absolute_error: 7.5184 - mean_squared_error: 204.6493
Epoch 4/6
542954/542954 [==============================] - 43s 80us/step - loss: 7.3761 - mean_absolute_error: 7.3761 - mean_squared_error: 196.7061
Epoch 5/6
542954/542954 [==============================] - 43s 80us/step - loss: 7.2721 - mean_absolute_error: 7.2721 - mean_squared_error: 191.5523
Epoch 6/6
67876/67876 [==============================] - 4s 55us/step
XVAL MAE: 6.8486833875029935


In [87]:
# lol NOPE. What about less dropout?
model = Sequential()
model.add(Dense(600, activation="sigmoid", input_dim=426))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, epochs=6, batch_size=256)
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")

Epoch 1/6
542954/542954 [==============================] - 45s 83us/step - loss: 8.7379 - mean_absolute_error: 8.7379 - mean_squared_error: 298.2580
Epoch 2/6
542954/542954 [==============================] - 41s 75us/step - loss: 7.5712 - mean_absolute_error: 7.5712 - mean_squared_error: 215.7863
Epoch 3/6
542954/542954 [==============================] - 41s 76us/step - loss: 7.2834 - mean_absolute_error: 7.2834 - mean_squared_error: 196.5177
Epoch 4/6
542954/542954 [==============================] - 42s 78us/step - loss: 7.1396 - mean_absolute_error: 7.1396 - mean_squared_error: 187.9118
Epoch 5/6
542954/542954 [==============================] - 42s 77us/step - loss: 7.0345 - mean_absolute_error: 7.0345 - mean_squared_error: 181.5814
Epoch 6/6
67876/67876 [==============================] - 4s 56us/step
XVAL MAE: 6.733853124014033


In [88]:

model = Sequential()
model.add(Dense(600, activation="sigmoid", input_dim=426))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, epochs=8, batch_size=256)
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")

Epoch 1/8
542954/542954 [==============================] - 46s 86us/step - loss: 8.3457 - mean_absolute_error: 8.3457 - mean_squared_error: 263.4149
Epoch 2/8
542954/542954 [==============================] - 44s 81us/step - loss: 7.3332 - mean_absolute_error: 7.3332 - mean_squared_error: 196.4161
Epoch 3/8
542954/542954 [==============================] - 44s 81us/step - loss: 7.1090 - mean_absolute_error: 7.1090 - mean_squared_error: 183.8373
Epoch 4/8
542954/542954 [==============================] - 44s 80us/step - loss: 6.9679 - mean_absolute_error: 6.9679 - mean_squared_error: 175.7016
Epoch 5/8
542954/542954 [==============================] - 42s 77us/step - loss: 6.8636 - mean_absolute_error: 6.8636 - mean_squared_error: 170.0832
Epoch 6/8
542954/542954 [==============================] - 42s 77us/step - loss: 6.7946 - mean_absolute_error: 6.7946 - mean_squared_error: 166.8979
Epoch 7/8
542954/542954 [==============================] - 42s 77us/step - loss: 6.7455 - mean_absolute_er

In [89]:
model.fit(X_train, y_train, epochs=2, batch_size=256)

Epoch 1/2
542954/542954 [==============================] - 43s 79us/step - loss: 6.6655 - mean_absolute_error: 6.6655 - mean_squared_error: 160.6850
Epoch 2/2
542954/542954 [==============================] - 43s 80us/step - loss: 6.6320 - mean_absolute_error: 6.6320 - mean_squared_error: 158.8025


In [90]:
model.fit(X_train, y_train, epochs=3, batch_size=256)

Epoch 1/3
542954/542954 [==============================] - 43s 79us/step - loss: 6.6067 - mean_absolute_error: 6.6067 - mean_squared_error: 158.1593
Epoch 2/3
542954/542954 [==============================] - 42s 78us/step - loss: 6.5896 - mean_absolute_error: 6.5896 - mean_squared_error: 157.3695
Epoch 3/3
542954/542954 [==============================] - 43s 79us/step - loss: 6.5586 - mean_absolute_error: 6.5586 - mean_squared_error: 155.9653


In [91]:
model.fit(X_train, y_train, epochs=2, batch_size=256)

Epoch 1/2
542954/542954 [==============================] - 43s 79us/step - loss: 6.5446 - mean_absolute_error: 6.5446 - mean_squared_error: 155.5750
Epoch 2/2
542954/542954 [==============================] - 43s 80us/step - loss: 6.5193 - mean_absolute_error: 6.5193 - mean_squared_error: 154.3072


In [92]:
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")

67876/67876 [==============================] - 2s 34us/step
XVAL MAE: 6.399070236831684


In [93]:
model.fit(X_train, y_train, epochs=4, batch_size=256)

Epoch 1/4
542954/542954 [==============================] - 43s 79us/step - loss: 6.5035 - mean_absolute_error: 6.5035 - mean_squared_error: 153.8057
Epoch 2/4
542954/542954 [==============================] - 43s 80us/step - loss: 6.4824 - mean_absolute_error: 6.4824 - mean_squared_error: 153.1254
Epoch 3/4
542954/542954 [==============================] - 45s 82us/step - loss: 6.4722 - mean_absolute_error: 6.4722 - mean_squared_error: 152.6193
Epoch 4/4
542954/542954 [==============================] - 45s 83us/step - loss: 6.4540 - mean_absolute_error: 6.4540 - mean_squared_error: 151.9311


In [95]:
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")

model.fit(X_train, y_train, epochs=4, batch_size=256)

67876/67876 [==============================] - 2s 33us/step
XVAL MAE: 6.331358876153304
Epoch 1/4
542954/542954 [==============================] - 43s 78us/step - loss: 6.4416 - mean_absolute_error: 6.4416 - mean_squared_error: 151.2298
Epoch 2/4
542954/542954 [==============================] - 44s 82us/step - loss: 6.4304 - mean_absolute_error: 6.4304 - mean_squared_error: 150.9122
Epoch 3/4
542954/542954 [==============================] - 45s 82us/step - loss: 6.4164 - mean_absolute_error: 6.4164 - mean_squared_error: 150.0803
Epoch 4/4
542954/542954 [==============================] - 43s 79us/step - loss: 6.4023 - mean_absolute_error: 6.4023 - mean_squared_error: 149.4390


In [96]:
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")


67876/67876 [==============================] - 2s 32us/step
XVAL MAE: 6.4226108718982


In [97]:
#best fit
model = Sequential()
model.add(Dense(600, activation="sigmoid", input_dim=426))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, epochs=15, batch_size=256)
xval_mae = model.evaluate(X_xval, y_xval, batch_size=128)[0]
print(f"XVAL MAE: {xval_mae}")

Epoch 1/15
542954/542954 [==============================] - 45s 82us/step - loss: 8.3531 - mean_absolute_error: 8.3531 - mean_squared_error: 264.6620
Epoch 2/15
542954/542954 [==============================] - 42s 77us/step - loss: 7.3503 - mean_absolute_error: 7.3503 - mean_squared_error: 197.5880
Epoch 3/15
542954/542954 [==============================] - 41s 76us/step - loss: 7.1092 - mean_absolute_error: 7.1092 - mean_squared_error: 183.6092
Epoch 4/15
542954/542954 [==============================] - 41s 76us/step - loss: 6.9605 - mean_absolute_error: 6.9605 - mean_squared_error: 175.3608
Epoch 5/15
542954/542954 [==============================] - 41s 76us/step - loss: 6.8583 - mean_absolute_error: 6.8583 - mean_squared_error: 169.7260
Epoch 6/15
542954/542954 [==============================] - 41s 76us/step - loss: 6.7917 - mean_absolute_error: 6.7917 - mean_squared_error: 166.0013
Epoch 7/15
542954/542954 [==============================] - 42s 78us/step - loss: 6.7328 - mean_abso

In [100]:
preds = model.predict(X_xval)
with open("../predictions/nn_15min_xval.txt", 'wt') as out:
    for pred in preds:
        out.write(str(pred)+'\n')


In [104]:
# Ok. Produce predictions from the best model so far, also fit and predict on :
# combined, 30 min (also re-scale model)
train, xval = pd.read_csv('../combined_data/30min/train.tsv.gz', sep='\t'), pd.read_csv("../combined_data/30min/xval.tsv.gz", sep='\t')

X_NUM_COLS = [
    'orca_total', 'frac_disabled', 'frac_youth', 'frac_senior', 'frac_li', 'frac_uw'
]
X_CAT_COLS = ['is_ns', 'is_rapid', 'is_weekend', 'trip_start_hr_30', 'rte', 'dir', 'day_of_week', 'region', 'start', 'end', 'summer']


label_encoders = {col: LabelEncoder() for col in X_CAT_COLS}
one_hot_encoder = OneHotEncoder()
scaler = StandardScaler()

X_train = np.concatenate((
    scaler.fit_transform(train[X_NUM_COLS]),
    one_hot_encoder.fit_transform(
        np.stack([label_encoders[col].fit_transform(train[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)
X_xval = np.concatenate((
    scaler.transform(xval[X_NUM_COLS]),
    one_hot_encoder.transform(
        np.stack([label_encoders[col].transform(xval[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)

y_train = train['ons']
y_xval = xval['ons']

column_labels = list()
column_labels.extend(X_NUM_COLS)
for cat in X_CAT_COLS:
    for clazz in label_encoders[cat].classes_:
        column_labels.append(f'{cat}: {clazz}')

#best fit
model = Sequential()
model.add(Dense(512, activation="sigmoid", input_dim=378))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, validation_data=(X_xval, y_xval), epochs=15, batch_size=256)

Epoch 1/15
424373/424373 [==============================] - 32s 76us/step - loss: 10.6505 - mean_absolute_error: 10.6505 - mean_squared_error: 531.3505
Epoch 2/15
424373/424373 [==============================] - 28s 67us/step - loss: 9.1163 - mean_absolute_error: 9.1163 - mean_squared_error: 342.5813
Epoch 3/15
424373/424373 [==============================] - 29s 68us/step - loss: 8.7286 - mean_absolute_error: 8.7286 - mean_squared_error: 303.7920
Epoch 4/15
424373/424373 [==============================] - 29s 68us/step - loss: 8.4910 - mean_absolute_error: 8.4910 - mean_squared_error: 282.5649
Epoch 5/15
424373/424373 [==============================] - 28s 67us/step - loss: 8.3449 - mean_absolute_error: 8.3449 - mean_squared_error: 269.7534
Epoch 6/15
424373/424373 [==============================] - 28s 67us/step - loss: 8.2425 - mean_absolute_error: 8.2425 - mean_squared_error: 262.1305
Epoch 7/15
424373/424373 [==============================] - 28s 66us/step - loss: 8.1556 - mean_ab

In [105]:
preds = model.predict(X_xval)
with open("../predictions/nn_30min_xval.txt", 'wt') as out:
    for pred in preds:
        out.write(str(pred)+'\n')



In [111]:
# winter only, 15 min
train, xval = pd.read_csv('../winter_data/aggregates/15min/train.tsv.gz', sep='\t'), pd.read_csv("../winter_data/aggregates/15min/xval.tsv.gz", sep='\t')

X_NUM_COLS = [
    'orca_total', 'frac_disabled', 'frac_youth', 'frac_senior', 'frac_li', 'frac_uw'
]
X_CAT_COLS = ['is_ns', 'is_rapid', 'is_weekend', 'trip_start_hr_15', 'rte', 'dir', 'day_of_week', 'region', 'start', 'end', 'summer']


label_encoders = {col: LabelEncoder() for col in X_CAT_COLS}
one_hot_encoder = OneHotEncoder()
scaler = StandardScaler()

X_train = np.concatenate((
    scaler.fit_transform(train[X_NUM_COLS]),
    one_hot_encoder.fit_transform(
        np.stack([label_encoders[col].fit_transform(train[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)
X_xval = np.concatenate((
    scaler.transform(xval[X_NUM_COLS]),
    one_hot_encoder.transform(
        np.stack([label_encoders[col].transform(xval[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)

y_train = train['ons']
y_xval = xval['ons']

column_labels = list()
column_labels.extend(X_NUM_COLS)
for cat in X_CAT_COLS:
    for clazz in label_encoders[cat].classes_:
        column_labels.append(f'{cat}: {clazz}')

#less aggressive fit for more homogenous data
model = Sequential()
model.add(Dense(600, activation="sigmoid", input_dim=425))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, validation_data=(X_xval, y_xval), epochs=5, batch_size=256)


Train on 225708 samples, validate on 28498 samples
Epoch 1/5
225708/225708 [==============================] - 23s 104us/step - loss: 8.3027 - mean_absolute_error: 8.3027 - mean_squared_error: 304.4028 - val_loss: 7.0636 - val_mean_absolute_error: 7.0636 - val_mean_squared_error: 204.8016
Epoch 2/5
225708/225708 [==============================] - 18s 78us/step - loss: 6.9415 - mean_absolute_error: 6.9415 - mean_squared_error: 202.4186 - val_loss: 6.6721 - val_mean_absolute_error: 6.6721 - val_mean_squared_error: 192.4184
Epoch 3/5
225708/225708 [==============================] - 18s 80us/step - loss: 6.7348 - mean_absolute_error: 6.7348 - mean_squared_error: 187.2427 - val_loss: 6.5133 - val_mean_absolute_error: 6.5133 - val_mean_squared_error: 177.3020
Epoch 4/5
225708/225708 [==============================] - 18s 78us/step - loss: 6.5926 - mean_absolute_error: 6.5926 - mean_squared_error: 176.4678 - val_loss: 6.4085 - val_mean_absolute_error: 6.4085 - val_mean_squared_error: 163.3112


In [112]:
model.fit(X_train, y_train, validation_data=(X_xval, y_xval), epochs=2, batch_size=256)

Train on 225708 samples, validate on 28498 samples
Epoch 1/2
225708/225708 [==============================] - 18s 81us/step - loss: 6.3970 - mean_absolute_error: 6.3970 - mean_squared_error: 161.3225 - val_loss: 6.2226 - val_mean_absolute_error: 6.2226 - val_mean_squared_error: 160.3527
Epoch 2/2
225708/225708 [==============================] - 18s 78us/step - loss: 6.3398 - mean_absolute_error: 6.3398 - mean_squared_error: 157.8516 - val_loss: 6.2047 - val_mean_absolute_error: 6.2047 - val_mean_squared_error: 159.7342


In [113]:
model.fit(X_train, y_train, validation_data=(X_xval, y_xval), epochs=2, batch_size=256)

Train on 225708 samples, validate on 28498 samples
Epoch 1/2
225708/225708 [==============================] - 19s 83us/step - loss: 6.2932 - mean_absolute_error: 6.2932 - mean_squared_error: 154.7478 - val_loss: 6.1522 - val_mean_absolute_error: 6.1522 - val_mean_squared_error: 153.4196
Epoch 2/2
225708/225708 [==============================] - 19s 86us/step - loss: 6.2447 - mean_absolute_error: 6.2447 - mean_squared_error: 151.8267 - val_loss: 6.1855 - val_mean_absolute_error: 6.1855 - val_mean_squared_error: 146.3405


In [114]:
preds = model.predict(X_xval)
with open("../predictions/nn_15min_winter_xval.txt", 'wt') as out:
    for pred in preds:
        out.write(str(pred)+'\n')


In [116]:
# winter only, 30 min
train, xval = pd.read_csv('../winter_data/aggregates/30min/train.tsv.gz', sep='\t'), pd.read_csv("../winter_data/aggregates/30min/xval.tsv.gz", sep='\t')

X_NUM_COLS = [
    'orca_total', 'frac_disabled', 'frac_youth', 'frac_senior', 'frac_li', 'frac_uw'
]
X_CAT_COLS = ['is_ns', 'is_rapid', 'is_weekend', 'trip_start_hr_30', 'rte', 'dir', 'day_of_week', 'region', 'start', 'end', 'summer']


label_encoders = {col: LabelEncoder() for col in X_CAT_COLS}
one_hot_encoder = OneHotEncoder()
scaler = StandardScaler()

X_train = np.concatenate((
    scaler.fit_transform(train[X_NUM_COLS]),
    one_hot_encoder.fit_transform(
        np.stack([label_encoders[col].fit_transform(train[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)
X_xval = np.concatenate((
    scaler.transform(xval[X_NUM_COLS]),
    one_hot_encoder.transform(
        np.stack([label_encoders[col].transform(xval[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)

y_train = train['ons']
y_xval = xval['ons']

column_labels = list()
column_labels.extend(X_NUM_COLS)
for cat in X_CAT_COLS:
    for clazz in label_encoders[cat].classes_:
        column_labels.append(f'{cat}: {clazz}')

#best fit
model = Sequential()
model.add(Dense(512, activation="sigmoid", input_dim=377))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, validation_data=(X_xval, y_xval), epochs=15, batch_size=256)

Train on 175074 samples, validate on 22386 samples
Epoch 1/15
175074/175074 [==============================] - 18s 100us/step - loss: 11.1047 - mean_absolute_error: 11.1047 - mean_squared_error: 700.7879 - val_loss: 9.0730 - val_mean_absolute_error: 9.0730 - val_mean_squared_error: 464.1272
Epoch 2/15
175074/175074 [==============================] - 12s 70us/step - loss: 8.6619 - mean_absolute_error: 8.6619 - mean_squared_error: 418.5517 - val_loss: 8.4057 - val_mean_absolute_error: 8.4057 - val_mean_squared_error: 390.2757
Epoch 3/15
175074/175074 [==============================] - 13s 72us/step - loss: 8.3264 - mean_absolute_error: 8.3264 - mean_squared_error: 371.3515 - val_loss: 8.0818 - val_mean_absolute_error: 8.0818 - val_mean_squared_error: 357.5992
Epoch 4/15
175074/175074 [==============================] - 13s 76us/step - loss: 8.0816 - mean_absolute_error: 8.0816 - mean_squared_error: 334.2666 - val_loss: 7.8811 - val_mean_absolute_error: 7.8811 - val_mean_squared_error: 324

KeyboardInterrupt: 

In [117]:
preds = model.predict(X_xval)
with open("../predictions/nn_30min_winter_xval.txt", 'wt') as out:
    for pred in preds:
        out.write(str(pred)+'\n')


In [121]:
# summer only, 15 min
train, xval = pd.read_csv('../summer_data/aggregates/15min/train.tsv.gz', sep='\t'), pd.read_csv("../summer_data/aggregates/15min/xval.tsv.gz", sep='\t')

X_NUM_COLS = [
    'orca_total', 'frac_disabled', 'frac_youth', 'frac_senior', 'frac_li', 'frac_uw'
]
X_CAT_COLS = ['is_ns', 'is_rapid', 'is_weekend', 'trip_start_hr_15', 'rte', 'dir', 'day_of_week', 'region', 'start', 'end', 'summer']


label_encoders = {col: LabelEncoder() for col in X_CAT_COLS}
one_hot_encoder = OneHotEncoder()
scaler = StandardScaler()

X_train = np.concatenate((
    scaler.fit_transform(train[X_NUM_COLS]),
    one_hot_encoder.fit_transform(
        np.stack([label_encoders[col].fit_transform(train[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)
X_xval = np.concatenate((
    scaler.transform(xval[X_NUM_COLS]),
    one_hot_encoder.transform(
        np.stack([label_encoders[col].transform(xval[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)

y_train = train['ons']
y_xval = xval['ons']

column_labels = list()
column_labels.extend(X_NUM_COLS)
for cat in X_CAT_COLS:
    for clazz in label_encoders[cat].classes_:
        column_labels.append(f'{cat}: {clazz}')

#less aggressive fit for more homogenous data
model = Sequential()
model.add(Dense(600, activation="sigmoid", input_dim=416))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, validation_data=(X_xval, y_xval), epochs=10, batch_size=256)


Train on 316444 samples, validate on 39948 samples
Epoch 1/10
316444/316444 [==============================] - 30s 96us/step - loss: 9.8631 - mean_absolute_error: 9.8631 - mean_squared_error: 358.1413 - val_loss: 8.7822 - val_mean_absolute_error: 8.7822 - val_mean_squared_error: 275.1887
Epoch 2/10
316444/316444 [==============================] - 26s 81us/step - loss: 8.4541 - mean_absolute_error: 8.4541 - mean_squared_error: 261.1516 - val_loss: 8.0937 - val_mean_absolute_error: 8.0937 - val_mean_squared_error: 240.4972
Epoch 3/10
316444/316444 [==============================] - 27s 84us/step - loss: 8.0472 - mean_absolute_error: 8.0472 - mean_squared_error: 235.4908 - val_loss: 7.8639 - val_mean_absolute_error: 7.8639 - val_mean_squared_error: 224.0024
Epoch 4/10
316444/316444 [==============================] - 26s 82us/step - loss: 7.8565 - mean_absolute_error: 7.8565 - mean_squared_error: 223.1352 - val_loss: 7.7216 - val_mean_absolute_error: 7.7216 - val_mean_squared_error: 207.87

KeyboardInterrupt: 

In [122]:
preds = model.predict(X_xval)
with open("../predictions/nn_15min_summer_xval.txt", 'wt') as out:
    for pred in preds:
        out.write(str(pred)+'\n')


In [125]:
# summer only, 30 min
train, xval = pd.read_csv('../summer_data/aggregates/30min/train.tsv.gz', sep='\t'), pd.read_csv("../summer_data/aggregates/30min/xval.tsv.gz", sep='\t')

X_NUM_COLS = [
    'orca_total', 'frac_disabled', 'frac_youth', 'frac_senior', 'frac_li', 'frac_uw'
]
X_CAT_COLS = ['is_ns', 'is_rapid', 'is_weekend', 'trip_start_hr_30', 'rte', 'dir', 'day_of_week', 'region', 'start', 'end', 'summer']


label_encoders = {col: LabelEncoder() for col in X_CAT_COLS}
one_hot_encoder = OneHotEncoder()
scaler = StandardScaler()

X_train = np.concatenate((
    scaler.fit_transform(train[X_NUM_COLS]),
    one_hot_encoder.fit_transform(
        np.stack([label_encoders[col].fit_transform(train[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)
X_xval = np.concatenate((
    scaler.transform(xval[X_NUM_COLS]),
    one_hot_encoder.transform(
        np.stack([label_encoders[col].transform(xval[col]) for col in X_CAT_COLS]).T
    ).todense()
), axis=1)

y_train = train['ons']
y_xval = xval['ons']

column_labels = list()
column_labels.extend(X_NUM_COLS)
for cat in X_CAT_COLS:
    for clazz in label_encoders[cat].classes_:
        column_labels.append(f'{cat}: {clazz}')

#best fit
model = Sequential()
model.add(Dense(512, activation="sigmoid", input_dim=368))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_absolute_error', 'mean_squared_error'])
model.fit(X_train, y_train, validation_data=(X_xval, y_xval), epochs=5, batch_size=256)

Train on 248579 samples, validate on 31495 samples
Epoch 1/5
248579/248579 [==============================] - 23s 94us/step - loss: 12.8144 - mean_absolute_error: 12.8144 - mean_squared_error: 737.2911 - val_loss: 15.8556 - val_mean_absolute_error: 15.8556 - val_mean_squared_error: 913.4162
Epoch 2/5
248579/248579 [==============================] - 18s 71us/step - loss: 10.8740 - mean_absolute_error: 10.8740 - mean_squared_error: 508.2829 - val_loss: 12.7028 - val_mean_absolute_error: 12.7028 - val_mean_squared_error: 452.7576
Epoch 3/5
248579/248579 [==============================] - 18s 72us/step - loss: 10.2558 - mean_absolute_error: 10.2558 - mean_squared_error: 431.8868 - val_loss: 11.3673 - val_mean_absolute_error: 11.3673 - val_mean_squared_error: 471.2871
Epoch 4/5
248579/248579 [==============================] - 18s 74us/step - loss: 9.8896 - mean_absolute_error: 9.8896 - mean_squared_error: 389.9734 - val_loss: 10.8307 - val_mean_absolute_error: 10.8307 - val_mean_squared_err

In [126]:
preds = model.predict(X_xval)
with open("../predictions/nn_30min_summer_xval.txt", 'wt') as out:
    for pred in preds:
        out.write(str(pred)+'\n')